In [157]:
import pandas as pd
df = pd.read_csv("train.csv")

In [158]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [159]:
y = df['Survived']
assert len(y) == len(y[y == 0]) + len(y[y==1])

In [182]:
df.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [161]:
# 可能有关特征为 Pclass Sex Age Fare Cabin
X = df.loc[:, ['Pclass', 'Sex', 'Age', 'Fare']]

In [162]:
# 将性别数值化
X.Sex[X['Sex'] == 'female'] = 1
X.Sex[X['Sex'] == 'male'] = 0

/Users/skyrover/miniconda3/envs/ml/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/skyrover/miniconda3/envs/ml/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [163]:
# 修复NaN数据
age_median = X['Age'].median()
X = X.fillna({'Age': age_median})
class_fare_median = {
    1: X[X['Pclass'] == 1]['Fare'].median(),
    2: X[X['Pclass'] == 2]['Fare'].median(),
    3: X[X['Pclass'] == 3]['Fare'].median()
}
# Fare为0或者NA的数据用 对应class的中位数
for row in X[(pd.isna(X['Fare'])) | (X['Fare'] == 0)].iterrows():
    p_class = row[1]['Pclass']
    row_id = row[0]
    X.iat[row_id, 3] = class_fare_median[p_class]

In [164]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=30)

In [165]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, random_state=2)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False)

In [166]:
forest.score(X_train, y_train), forest.score(X_test, y_test)

(0.9835329341317365, 0.8116591928251121)

In [167]:
forest.feature_importances_

array([0.09576782, 0.28209662, 0.29987816, 0.3222574 ])

In [168]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state=0, max_depth=5, learning_rate=0.1, n_estimators=100)
gbrt.fit(X_train, y_train)
gbrt.score(X_train, y_train), gbrt.score(X_test, y_test)

(0.9535928143712575, 0.8430493273542601)

In [183]:
df = pd.read_csv("test.csv")

In [184]:
X = df.loc[:, ['Pclass', 'Sex', 'Age', 'Fare']]

In [185]:
# 将性别数值化
X.Sex[X['Sex'] == 'female'] = 1
X.Sex[X['Sex'] == 'male'] = 0

/Users/skyrover/miniconda3/envs/ml/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/skyrover/miniconda3/envs/ml/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [186]:
# 修复NaN数据
age_median = X['Age'].median()
X = X.fillna({'Age': age_median})
class_fare_median = {
    1: X[X['Pclass'] == 1]['Fare'].median(),
    2: X[X['Pclass'] == 2]['Fare'].median(),
    3: X[X['Pclass'] == 3]['Fare'].median()
}
# Fare为0或者NA的数据用 对应class的中位数
for row in X[(pd.isna(X['Fare'])) | (X['Fare'] == 0)].iterrows():
    p_class = row[1]['Pclass']
    row_id = row[0]
    X.iat[row_id, 3] = class_fare_median[p_class]

In [187]:
y_test = forest.predict(X)
output = pd.DataFrame({'PassengerId': df['PassengerId'], 'Survived': y_test})
output.to_csv('submission.csv', index=False)

In [188]:
y_test = gbrt.predict(X)
output = pd.DataFrame({'PassengerId': df['PassengerId'], 'Survived': y_test})
output.to_csv('submission.csv', index=False)

In [190]:
gbrt

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)